## Part 1: Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [20]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [21]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [22]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [23]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)



In [24]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
le = LabelEncoder()
X_train['OverTime'] = le.fit_transform(X_train['OverTime'])
X_test['OverTime'] = le.transform(X_test['OverTime'])
X_train['OverTime'].value_counts()

OverTime
0    780
1    322
Name: count, dtype: int64

In [25]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [26]:
# Create a OneHotEncoder for the Department column
ohe_dep = OneHotEncoder()

# Fit the encoder to the training data
ohe_dep.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dep_encoded = ohe_dep.transform(y_train[['Department']]).toarray()
y_test_dep_encoded = ohe_dep.transform(y_test[['Department']]).toarray()




In [27]:
# Create a OneHotEncoder for the Attrition column
ohe_att = OneHotEncoder()

# Fit the encoder to the training data
ohe_att.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_att_encoded = ohe_att.transform(y_train[['Attrition']]).toarray()
y_test_att_encoded = ohe_att.transform(y_test[['Attrition']]).toarray()


## Create, Compile, and Train the Model

In [28]:
# Find the number of columns in the X training data
num_columns = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [29]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_dep_encoded.shape[1], activation='softmax', name='department_output')(department_hidden_layer)


In [30]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_test_att_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_hidden_layer)


In [31]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
               loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'}, 
               metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │        768 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 16)        │        528 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 16)        │        528 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_7[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,989 (15.58 KB)

 Trainable params: 3,989 (15.58 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Train the model
model.fit(
    X_train_scaled, 
    {'department_output': y_train_dep_encoded, 'attrition_output': y_train_att_encoded},
    epochs=100,
    batch_size=32,
    validation_data=(X_test_scaled, {'department_output': y_test_dep_encoded, 'attrition_output': y_test_att_encoded}),
    verbose=1
)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - attrition_output_accuracy: 0.4684 - department_output_accuracy: 0.3597 - loss: 1.8290 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.4634
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8232 - department_output_accuracy: 0.6471 - loss: 1.4177 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.2469
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8379 - department_output_accuracy: 0.6667 - loss: 1.2143 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.1939
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8006 - department_output_accuracy: 0.6476 - loss: 1.2530 - val_attrition_output_accuracy: 0.8696 - val_department_output_accuracy: 0.6522 - val_loss: 1.1811
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1m

In [33]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_test_dep_encoded, 'attrition_output': y_test_att_encoded})  
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - attrition_output_accuracy: 0.8446 - department_output_accuracy: 0.6039 - loss: 2.2750


[2.08872652053833, 0.85597825050354, 0.5896739363670349]

In [34]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[2]*100:.2f}%")
print(f"Attrition Accuracy: {test_results[1]*100:.2f}%")

Department Accuracy: 58.97%
Attrition Accuracy: 85.60%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, accuracy isn't the best metric due to class imbalances. Precision, recall, and F1-score provide better insights, especially for minority classes.

2. Softmax for both output layers. It ensures outputs sum to 1 and represent probabilities for each class, facilitating better decision-making.

3. Some ways to possibly improve the model are:
* Increase training data for better generalization
* Add/modify features through feature engineering
* Hyperparameter tuning for optimal learning
* Regularization to prevent overfitting
* Experimenting with different architectures/models
